In [32]:
import glob
import pandas as pd

rawData = []
outfile = "results.pkl"

In [33]:
for file in glob.glob("*/*.log"):
    print(file)
    workloadParts = (file.split("/")[1]).split("_")
    platform = workloadParts[0]
    functionType = (workloadParts[1])
    regression = int(workloadParts[2])
    #    with open(file) as logfile:
    with open(file) as logfile:
        for line in logfile:
            # for DuetFaaS
            if line.__contains__("Response Time"):
                f1Durations = []
                f2Durations = []
                line = line.strip()
                parts = line.split(" ")
                print(parts)
                if "." in parts[4]:
                    f1 = int(parts[4].split(".")[0])
                else:
                    f1 = int(parts[4][0:parts[4].index("m")])
                if "." in parts[18]:
                    f2 = int(parts[18].split(".")[0])
                else:
                    f2 = int(parts[18][0:parts[18].index("m")])
                #print(f" f1 is {f1}, f2 is {f2}")
                f1Durations.append(f1)
                f2Durations.append(f2)
                row = {}
                row["platform"] = "aws"
                row["functionType"] = functionType
                row["regression"] = regression
                row["mode"] = "duet"
                #print(f"platform is {platform}, functionType is {functionType}, regression is {regression}")
                for i in range(0, len(f1Durations)):
                    row["f1"] = f1Durations[i]
                    row["f2"] = f2Durations[i]
                    row["change"] = f2Durations[i] / f1Durations[i]
                    rawData.append(row)
            # for faasterBench
            if line.__contains__("faaster"):
                if(file.__contains__("aws")):
                    line = line[:-4]
                    if line.__contains__("modeA") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeA"

                        f1Durations = []
                        f2Durations = []
                        f2Found = False

                        for i in range(4,len(parts2)):
                            if parts2[i] == "f2":
                                break
                            
                            #if int(parts2[i]) < 2000 and int(parts2[i+4]) < 2100:
                            f1Durations.append(int(parts2[i]))
                            f2Durations.append(int(parts2[i+4]))
                        for i in range(0, len(f1Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = f1Durations[i]
                            row["f2"] = f2Durations[i]
                            row["change"] = f2Durations[i] / f1Durations[i]
                            rawData.append(row)
                    if line.__contains__("modeB") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeB"
                        f1Durations = []

                        for i in range(4,len(parts2)):
                            if parts2[i] == "f2":
                                break
                            else:
                                if int(parts2[i]) < 2000:
                                    f1Durations.append(int(parts2[i]))

                        for i in range(0, len(f1Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = f1Durations[i]
                            row["f2"] = 0
                            row["change"] = 0
                            rawData.append(row)

                    if line.__contains__("modeC") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeC"
                        f2Durations = []

                        for i in range(5,len(parts2)):
                            if parts2[i] == "\",":
                                break
                            else:
                                if int(parts2[i]) < 2100:
                                    f2Durations.append(int(parts2[i]))

                        for i in range(0, len(f2Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = 0
                            row["f2"] = f2Durations[i]
                            row["change"] = 0
                            rawData.append(row)
                if(file.__contains__("google")):
                    if line.__contains__("textPayload"):
                        if line.__contains__("modeA") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeA"

                            f1Durations = []
                            f2Durations = []
                            f2Found = False

                            for i in range(4,len(parts2)):
                                if parts2[i] == "f2":
                                    break
                                
                                else:
                                    if int(parts2[i]) < 1800 and int(parts2[i+4]) < 1800:
                                        f1Durations.append(int(parts2[i]))
                                        f2Durations.append(int(parts2[i+4]))

                            for i in range(0, len(f1Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = f1Durations[i]
                                row["f2"] = f2Durations[i]
                                row["change"] = f2Durations[i] / f1Durations[i]
                                rawData.append(row)
                        if line.__contains__("modeB") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeB"

                            f1Durations = []

                            for i in range(4,len(parts2)):
                                if parts2[i] == "f2":
                                    break
                                else:
                                    if int(parts2[i]) < 1800:
                                        f1Durations.append(int(parts2[i]))

                            for i in range(0, len(f1Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = f1Durations[i]
                                row["f2"] = 0
                                row["change"] = 0
                                rawData.append(row)

                        if line.__contains__("modeC") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeC"

                            f2Durations = []

                            for i in range(5,len(parts2)):
                                if parts2[i] == "\",":
                                    break
                                else:
                                    if int(parts2[i]) < 1800:
                                        f2Durations.append(int(parts2[i]))

                            for i in range(0, len(f2Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = 0
                                row["f2"] = f2Durations[i]
                                row["change"] = 0
                                rawData.append(row)



duet/duet_mem_5_04_19_11.log
['{"level":"info","message":"A', '-', 'Response', 'Time:', '73.73494600000049ms,', 'processId:', '20,', 'isColdStart:', 'false,', 'idlingInQueue:', '0ms,', 'requestReceivedAt:', '2024-04-19T11:37:04.389Z', '|', 'B', '-', 'Response', 'Time:', '63.891654999999446ms,', 'processId:', '26,', 'isColdStart:', 'false,', 'idlingInQueue:', '0ms,', 'requestReceivedAt:', '2024-04-19T11:37:04.409Z","timestamp":"2024-04-19', '11:37:04.531"}']
['{"level":"info","message":"A', '-', 'Response', 'Time:', '68.0080999999991ms,', 'processId:', '26,', 'isColdStart:', 'false,', 'idlingInQueue:', '0ms,', 'requestReceivedAt:', '2024-04-19T11:37:05.206Z', '|', 'B', '-', 'Response', 'Time:', '66.90869999999995ms,', 'processId:', '20,', 'isColdStart:', 'false,', 'idlingInQueue:', '0ms,', 'requestReceivedAt:', '2024-04-19T11:37:05.182Z","timestamp":"2024-04-19', '11:37:05.323"}']
['{"level":"info","message":"A', '-', 'Response', 'Time:', '59.897361000003ms,', 'processId:', '26,', 'isCo

In [34]:
df_results = pd.DataFrame(rawData)
df_results.head()
df_results.describe()
print(df_results)
df_results.to_pickle(outfile)
#df_results.to_csv("nils2.csv")

      platform functionType  regression   mode   f1   f2    change
0          aws          mem           5   duet   73   63  0.863014
1          aws          mem           5   duet   68   66  0.970588
2          aws          mem           5   duet   59  102  1.728814
3          aws          mem           5   duet  105   61  0.580952
4          aws          mem           5   duet   67   63  0.940299
...        ...          ...         ...    ...  ...  ...       ...
14862      aws          mem           5  modeB   63    0  0.000000
14863      aws          mem           5  modeC    0   90  0.000000
14864      aws          mem           5  modeB  100    0  0.000000
14865      aws          mem           5  modeB   89    0  0.000000
14866      aws          mem           5  modeC    0  105  0.000000

[14867 rows x 7 columns]
